# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.common import ML_api
influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.
cpu is available.


# 1. set param from Front End

In [2]:
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        #step_split일 경우만 past_step과 future_step이 존재
        'data_clean_option' : "false"
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'step_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        'past_step':24, #step_split일 경우만 past_step과 future_step이 존재
        'future_step':2
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainX',
        "feature_list":['col_0', 'col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainy',
        "feature_list":['value']
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'window_split' # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
    },
    "model_info" :{
        'model_purpose' : 'classification',
        'model_method' : 'LSTM_cf',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True","num_classes":6, 'rnn_type':'lstm'}
    }
}

# 2. Backend

In [3]:
# 1. Backend Prameter Setting
from Clust.clust.ML.common import ML_api

# parameter tunning
param = param3
param = ML_api.convert_param_for_backend(param)

In [4]:
# 2. train data meta check
train_data_info = ML_api.get_train_data_meta (mongodb_client, param['ingestion_param_X'])
param['train_data_info'] = train_data_info

In [5]:
# 3. Data Preparation
#train_X_array, train_y_array, val_X_array, val_y_array = ML_api.ML_data_preparation(param, influxdb_client)

from Clust.clust.ML.common import ML_pipeline
# 1. Oirignla data ingestion
data_X, data_y = ML_pipeline.Xy_data_preparation(param['ingestion_param_X'], 
                                             param['data_y_flag'], 
                                             param['ingestion_param_y'],
                                             'ms_all', 
                                             influxdb_client)
# 2. Scaling
dataX_scaled, datay_scaled = ML_pipeline.Xy_data_scaling_train(param['ingestion_param_X']['ms_name'], 
                                                                                 data_X, 
                                                                                 param['ingestion_param_y']['ms_name'], 
                                                                                 data_y, 
                                                                                 param['scaler_param'])



# 3.clean column
dataX_scaled = ML_pipeline.clean_low_quality_column(dataX_scaled, param['transform_param'])

param['transform_param']

{'bucket_name': 'integration', 'ms_name': 'classification_actionPattern_cleanLevel0_trainX', 'feature_list': ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']}
['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']
Make New scaler File
./scaler/classification_actionPattern_cleanLevel0_trainX/minmax/2b219d77d5b069971addb3d7c728ec0d/scaler.pkl
['value']
Make New scaler File


{'data_clean_option': False,
 'split_mode': 'window_split',
 'nan_process_info': {'type': 'num',
  'ConsecutiveNanLimit': 10000,
  'totalNaNLimit': 100000},
 'max_nan_limit_ratio': 0.5}

In [6]:
# 4. split train/Val
split_ratio = 0.8
train_X, val_X, train_y, val_y, param['transform_param']= ML_pipeline.split_data_by_mode(dataX_scaled, 
                                                                                         datay_scaled, 
                                                                                         split_ratio, 
                                                                                         param['transform_param'])

{'data_clean_option': False, 'split_mode': 'window_split', 'nan_process_info': {'type': 'num', 'ConsecutiveNanLimit': 10000, 'totalNaNLimit': 100000}, 'max_nan_limit_ratio': 0.5, 'future_step': None, 'past_step': 128}
window_split
None


In [7]:
train_X

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8
time,,,,,,,,,
1992-01-11 00:00:00+00:00,0.486708,0.584184,0.584082,0.535918,0.515370,0.543161,0.555284,0.521129,0.596563
1992-01-11 00:11:15+00:00,0.490641,0.582380,0.583903,0.537439,0.513381,0.540716,0.559043,0.519822,0.597505
1992-01-11 00:22:30+00:00,0.490300,0.583392,0.581139,0.536537,0.516124,0.541288,0.558741,0.520847,0.596278
1992-01-11 00:33:45+00:00,0.488638,0.582772,0.581703,0.537067,0.514629,0.542166,0.557183,0.520517,0.597802
1992-01-11 00:45:00+00:00,0.490906,0.582191,0.578922,0.537820,0.514204,0.539202,0.559361,0.520218,0.596521
...,...,...,...,...,...,...,...,...,...
2008-02-16 23:03:45+00:00,0.487328,0.582224,0.561860,0.531996,0.508964,0.542586,0.557988,0.504818,0.568865
2008-02-16 23:15:00+00:00,0.486507,0.581658,0.561933,0.532397,0.508276,0.541986,0.557211,0.504345,0.568898
2008-02-16 23:26:15+00:00,0.487463,0.581152,0.563547,0.533299,0.508082,0.541137,0.558124,0.503923,0.570212


In [8]:
print(len(val_X)/128)
print(len(val_y))

1471.0
1471


In [9]:
# 5. Transform array style
train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(param['transform_param'], 
                                                                        train_X, 
                                                                        train_y)
val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(param['transform_param'], 
                                                                    val_X, 
                                                                    val_y)

window_size: 128 nan_limit_num: 64
(752768, 9) (5881, 128, 9)
(5881, 1) (5881, 1)
window_size: 128 nan_limit_num: 64
(188288, 9) (1471, 128, 9)
(1471, 1) (1471, 1)


In [10]:
# 4.Training
param = ML_api.ML_training(train_X_array,  train_y_array, val_X_array, val_y_array, param)

./Models/LSTM_cf/classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_/classification_actionPattern_cleanLevel0_trainX/model.pkl
Start training model

Epoch 1/5
train Loss: 0.8013 Acc: 0.7718
val Loss: 0.5130 Acc: 0.8001

Training complete in 0m 33s
Best val Acc: 0.975962


In [11]:
param

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'classification_actionPattern_cleanLevel0_trainX',
  'feature_list': ['col_0',
   'col_1',
   'col_2',
   'col_3',
   'col_4',
   'col_5',
   'col_6',
   'col_7',
   'col_8']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'classification_actionPattern_cleanLevel0_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/',
  'scaler_file_path': {'XScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/classification_actionPattern_cleanLevel0_trainX/minmax/2b219d77d5b069971addb3d7c728ec0d/scaler.pkl'},
   'yScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/classification_actionPattern_cleanLevel0_trainy/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'}}},
 'transform_param': {'data_clean_option': False,
  'split_mode': 'window_split',
  'nan_process_info': {'type': 'num',
 

In [12]:
# save meta
from Clust.clust.ML.tool import meta
meta_file_name = "./meta.json"
#ml_meta.save_model_meta_into_mongodb(mongodb_client, param, 'model','meta')
meta.save_model_meta_into_local(meta_file_name, param)

======== OK ========


200